In [43]:
import pandas as pd
from lxml import etree
import bs4 as bs
import requests
import time
import sys
from selenium import webdriver
import tqdm
import seaborn as sns
from nltk.corpus import stopwords

In [4]:
path = 'D:\\OneDrive - National University of Singapore\\industry-classification-master\\'

In [4]:
sector = pd.read_html('https://www.tradingview.com/markets/stocks-japan/sectorandindustry-sector/')[0].iloc[:,0]

In [5]:
sector = sector.values.tolist()
sector = list(map(lambda x: '-'.join(x.lower().split(' ')), sector))

In [6]:
name_list = []
for i in sector:
    sub_data = pd.read_html('https://www.tradingview.com/markets/stocks-japan/sectorandindustry-sector/%s/'%i)[0].iloc[:,0].values.tolist()
    sub_data = list(map(lambda x: x.split(' ')[0]+'.T' ,sub_data))
    name_list+=sub_data

In [8]:
def get_intro(ticker):
    url = 'https://finance.yahoo.com/quote/'+ticker+'/profile?p='+ticker
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')

    driver = webdriver.Chrome(options=options)

#driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    html = etree.HTML(html)

# page = requests.get(url)
# html = etree.HTML(page.text)

# items = html.xpath('//*[@id="Col1-0-Profile-Proxy"]/section/section[2]/p')
    items = html.xpath(
        '//*[@id="Col1-0-Profile-Proxy"]/section/section[2]/descendant-or-self::text()')
    item_2 = html.xpath(
        '//*[@id="Col1-0-Profile-Proxy"]/section/div[1]/div/div/p[2]/span[2]//text()')
    item_3 = html.xpath(
        '//*[@id="Col1-0-Profile-Proxy"]/section/div[1]/div/div/p[2]/span[4]//text()')
    driver.close()
    return [items[1],item_2[0],item_3[0]]

In [43]:
ticker=name_list[1]

In [9]:
intros=dict()
for ticker in tqdm.tqdm_notebook(name_list,desc='Companies fetched',unit='company'):
    try:
        intros[ticker]=get_intro(ticker)
    except:
        pass

d:\users\apply\anaconda3\envs\venv\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [10]:
intro_pd=pd.DataFrame({'company':list(intros.keys()),'intro':[ls[0] for ls in list(intros.values())],'segment':[ls[1] for ls in list(intros.values())],'industry':[ls[2] for ls in list(intros.values())]})
intro_pd['introLen']=intro_pd['intro'].apply(len)
intro_pd[intro_pd['introLen']<400].head()

,company,intro,segment,industry,introLen
10,2163.T,"Artner Co.,Ltd. provides worker dispatching an...",Industrials,Staffing & Employment Services,352
11,2164.T,"Chiikishinbunsha Co.,Ltd. publishes newspapers...",Communication Services,Publishing,225
12,2168.T,Pasona Group Inc. provides human resource-rela...,Industrials,Staffing & Employment Services,352
14,2170.T,Link and Motivation Inc. provides consulting s...,Industrials,Consulting Services,279
21,2193.T,Cookpad Inc. provides online recipe sharing an...,Communication Services,Advertising Agencies,371


In [11]:
intro_pd.to_csv('Japan intro.csv')

In [1]:
import pandas as pd
import pandas_datareader.data as web

d:\anaconda\envs\ml\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
intro_pd = pd.read_csv('Japan intro.csv')

In [18]:
stock_list = intro_pd['company'].values.tolist()
stock_data = web.DataReader(name=stock_list,start='20180101',end='20200430',data_source='yahoo')

d:\anaconda\envs\ml\lib\site-packages\pandas_datareader\base.py:270: SymbolWarning: Failed to read symbol: '1757.T', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
d:\anaconda\envs\ml\lib\site-packages\pandas_datareader\base.py:270: SymbolWarning: Failed to read symbol: '8139.T', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
d:\anaconda\envs\ml\lib\site-packages\pandas_datareader\base.py:270: SymbolWarning: Failed to read symbol: '2876.T', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
d:\anaconda\envs\ml\lib\site-packages\pandas_datareader\base.py:270: SymbolWarning: Failed to read symbol: '2417.T', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
d:\anaconda\envs\ml\lib\site-packages\pandas_datareader\base.py:270: SymbolWarning: Failed to read symbol: '3021.T', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
d:\anaconda\envs\ml\lib\site-packages\pandas_datareader\base.py:270: Symbol

In [19]:
stock_data.to_csv(path + 'data\\Japan price.csv')

In [21]:
#stock_data=stock_data.astype(float)
stock_data['Adj Close'] = stock_data['Adj Close'].pct_change()
stock_data = stock_data[['Adj Close']]
stock_data = stock_data.stack()
#stock_data = stock_data.loc[stock_data['Volume']!=0,:]
stock_data.dropna(inplace=True)
stock_data.reset_index(inplace=True)
stock_data.reset_index(drop=True,inplace=True)

In [22]:
stock_data.columns = ['date','company','return']

In [23]:
stock_data

,date,company,return
0,2018-01-02,1418.T,0.000000
1,2018-01-02,1436.T,0.000000
2,2018-01-02,2122.T,0.000000
3,2018-01-02,2124.T,0.000000
4,2018-01-02,2136.T,0.000000
...,...,...,...
1177110,2020-04-30,9536.T,0.016762
1177111,2020-04-30,9537.T,0.001600
1177112,2020-04-30,9539.T,0.042345
1177113,2020-04-30,9543.T,0.012579


In [29]:
import keras
from keras.optimizers import SGD
from keras import callbacks
from keras.initializers import VarianceScaling,lecun_normal,glorot_uniform
from keras.models import Model
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from clustering_class import *
from combine_models import *
from utils import *
from DEC import *
from generate_word_vector import *
%load_ext autoreload
%autoreload 2

In [30]:
des = pd.read_csv(path+'/Japan intro.csv',index_col=0)

In [44]:
all_data = {}
company_list=des['company'].values
desc_list=des['intro'].values
stopwords = stopwords.words('english')+[' ']
vector_generator=generate_doc_vector(company_list,desc_list,stopwords,0.8,5,'English')
bag_of_words=vector_generator.bag_of_words()
all_data['bag of words']=bag_of_words

In [50]:
all_data['bert'] = pd.read_csv(path+'data\\Japan bert.csv')

In [52]:
for i in range(50,401,50):
    doc2vec_1=vector_generator.doc2vector(vector_size=i,alpha=0.01)
    doc2vec_2=vector_generator.doc2vector(vector_size=i,alpha=0.01,dm=0)
    all_data['pvdm %d length'%i]=doc2vec_1
    all_data['pvdbow %d length'%i]=doc2vec_2

In [53]:
for i in range(200,1101,100):
    Lsi=vector_generator.LSI(num_topics=i)
    all_data['lsi %d length'%i]=Lsi

In [54]:
segment=des.segment
industry=des.industry
print('%d companys'%len(set(all_data['bert']['company'])))
print('%d segments'%len(set(segment)))
print('%d industries'%len(set(industry)))

2084 companys
11 segments
127 industries


In [55]:
def lr_sch(epoch):
    return 1/(epoch+1)

In [56]:
tol = 0.01
maxiter = 1000
update_interval = 100
n_clusters = 12
n_epochs=150
batch_size_auto=128
batch_size=64
init = glorot_uniform()
pretrain_optimizer = SGD(lr=1, momentum=0.9)
optimizer=SGD(lr=0.01,momentum=0.9,decay=0.01/n_epochs)
lr_scheduler=keras.callbacks.LearningRateScheduler(lr_sch, verbose=0)
early_stopping= keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0002, patience=5)
logdir="logs/fit/USBert" 
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
output=[]
kmeans = KMeans(n_clusters=n_clusters)

In [57]:
for key,input in all_data.items():
    input_x=input.iloc[:,1:]
    dims=[input_x.shape[-1],500,200,2000,50]
    autoencode, encoder = autoencoder(dims, act='relu', init=init, dropout=0.2)
    autoencode.compile(optimizer=pretrain_optimizer, loss='mse')
    autoencode.fit(input_x, input_x, batch_size=batch_size, epochs=n_epochs,callbacks=[lr_scheduler,early_stopping])
    kmeans=kmeans.fit(encoder.predict(input_x))
    y_init=kmeans.cluster_centers_
    loss = 0
    index = 0
    index_array = np.arange(input_x.shape[0])
    clustering_layer = ClusteringLayer(n_clusters=n_clusters, name='clustering',weights=[y_init])(encoder.output)
    model = Model(inputs=encoder.input, outputs=clustering_layer)    
    model.compile(loss='kld', optimizer=optimizer)
    y_pred_last=kmeans.predict(encoder.predict(input_x))
    for ite in range(maxiter):
        if ite % update_interval == 0:
            q = model.predict(input_x, verbose=0)
            p = target_distribution(q)  # update the auxiliary target distribution p

            # evaluate the clustering performance
            y_pred = q.argmax(1)

            # check stop criterion
            delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
            y_pred_last = np.copy(y_pred)
            print('the '+str(ite)+'th training on '+key)
            if ite > 0 and delta_label < tol:
                print('delta_label = ', delta_label, ' < %d'%tol)
                print('Reached tolerance threshold. Stopping training.')
                break
        idx = index_array[index * batch_size: min((index+1) * batch_size, input_x.shape[0])]
        loss = model.train_on_batch(x=input_x.iloc[idx], y=p[idx])
        index = index + 1 if (index + 1) * batch_size <= input_x.shape[0] else 0
    q = model.predict(input_x,verbose=0)
    clas=pd.DataFrame(np.argmax(q,1),columns=[key])
    clas.insert(0,'company',input['company'])
    output.append(clas)


Epoch 1/150
2084/2084 [==============================] - 2s 777us/step - loss: 0.0414
Epoch 2/150
 320/2084 [===>..........................] - ETA: 0s - loss: 0.015

d:\anaconda\envs\ml\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


2084/2084 [==============================] - 1s 411us/step - loss: 0.0108
Epoch 3/150
2084/2084 [==============================] - 1s 416us/step - loss: 0.0055
Epoch 4/150
2084/2084 [==============================] - 1s 433us/step - loss: 0.0040
Epoch 5/150
2084/2084 [==============================] - 1s 445us/step - loss: 0.0031
Epoch 6/150
2084/2084 [==============================] - 1s 442us/step - loss: 0.0026
Epoch 7/150
2084/2084 [==============================] - 1s 496us/step - loss: 0.0022
Epoch 8/150
2084/2084 [==============================] - 1s 456us/step - loss: 0.0020
Epoch 9/150
2084/2084 [==============================] - 1s 459us/step - loss: 0.0018
Epoch 10/150
2084/2084 [==============================] - 1s 452us/step - loss: 0.0017
Epoch 11/150
2084/2084 [==============================] - 1s 451us/step - loss: 0.0016
Epoch 12/150
2084/2084 [==============================] - 1s 451us/step - loss: 0.0015
Epoch 13/150
2084/2084 [==============================] - 1s 49

d:\anaconda\envs\ml\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


2084/2084 [==============================] - 1s 275us/step - loss: 0.0951
Epoch 3/150
2084/2084 [==============================] - 1s 319us/step - loss: 0.0779
Epoch 4/150
2084/2084 [==============================] - 1s 277us/step - loss: 0.0736
Epoch 5/150
2084/2084 [==============================] - 1s 264us/step - loss: 0.0707
Epoch 6/150
2084/2084 [==============================] - 1s 332us/step - loss: 0.0696
Epoch 7/150
2084/2084 [==============================] - 1s 279us/step - loss: 0.0671
Epoch 8/150
2084/2084 [==============================] - 1s 271us/step - loss: 0.0654
Epoch 9/150
2084/2084 [==============================] - 1s 276us/step - loss: 0.0649
Epoch 10/150
2084/2084 [==============================] - 1s 266us/step - loss: 0.0653
Epoch 11/150
2084/2084 [==============================] - 1s 269us/step - loss: 0.0646
Epoch 12/150
2084/2084 [==============================] - 1s 294us/step - loss: 0.0646
Epoch 13/150
2084/2084 [==============================] - 1s 30

...

2084/2084 [==============================] - 1s 342us/step - loss: 0.0057
Epoch 3/150
2084/2084 [==============================] - 1s 290us/step - loss: 0.0023
Epoch 4/150
2084/2084 [==============================] - 1s 294us/step - loss: 0.0018
Epoch 5/150
2084/2084 [==============================] - 1s 291us/step - loss: 0.0016
Epoch 6/150
2084/2084 [==============================] - 1s 292us/step - loss: 0.0015
Epoch 7/150
2084/2084 [==============================] - 1s 298us/step - loss: 0.0014
Epoch 8/150
2084/2084 [==============================] - 1s 292us/step - loss: 0.0014
Epoch 9/150
2084/2084 [==============================] - 1s 296us/step - loss: 0.0013
Epoch 10/150
2084/2084 [==============================] - 1s 293us/step - loss: 0.0013
Epoch 11/150
2084/2084 [==============================] - 1s 348us/step - loss: 0.0013
Epoch 12/150
2084/2084 [==============================] - 1s 296us/step - loss: 0.0012
Epoch 13/150
2084/2084 [==============================] - 1s 29

d:\anaconda\envs\ml\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


2084/2084 [==============================] - 1s 327us/step - loss: 0.0055
Epoch 3/150
2084/2084 [==============================] - 1s 301us/step - loss: 0.0025
Epoch 4/150
2084/2084 [==============================] - 1s 302us/step - loss: 0.0019
Epoch 5/150
2084/2084 [==============================] - 1s 303us/step - loss: 0.0017
Epoch 6/150
2084/2084 [==============================] - 1s 304us/step - loss: 0.0016
Epoch 7/150
2084/2084 [==============================] - 1s 299us/step - loss: 0.0015
Epoch 8/150
2084/2084 [==============================] - 1s 307us/step - loss: 0.0014
Epoch 9/150
2084/2084 [==============================] - 1s 304us/step - loss: 0.0014
Epoch 10/150
2084/2084 [==============================] - 1s 356us/step - loss: 0.0014
Epoch 11/150
2084/2084 [==============================] - 1s 306us/step - loss: 0.0013
Epoch 12/150
2084/2084 [==============================] - 1s 307us/step - loss: 0.0013
Epoch 13/150
2084/2084 [==============================] - 1s 30

d:\anaconda\envs\ml\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


2084/2084 [==============================] - 1s 322us/step - loss: 0.0074
Epoch 3/150
2084/2084 [==============================] - 1s 340us/step - loss: 0.0031
Epoch 4/150
2084/2084 [==============================] - 1s 339us/step - loss: 0.0022
Epoch 5/150
2084/2084 [==============================] - 1s 348us/step - loss: 0.0019
Epoch 6/150
2084/2084 [==============================] - 1s 347us/step - loss: 0.0017
Epoch 7/150
2084/2084 [==============================] - 1s 422us/step - loss: 0.0016
Epoch 8/150
2084/2084 [==============================] - 1s 468us/step - loss: 0.0015
Epoch 9/150
2084/2084 [==============================] - 1s 388us/step - loss: 0.0014
Epoch 10/150
2084/2084 [==============================] - 1s 365us/step - loss: 0.0014
Epoch 11/150
2084/2084 [==============================] - 1s 391us/step - loss: 0.0013
Epoch 12/150
2084/2084 [==============================] - 1s 376us/step - loss: 0.0013
Epoch 13/150
2084/2084 [==============================] - 1s 41

In [60]:
all_cluster.to_csv('Japan cluster.csv',index=False)

In [58]:
from sklearn.preprocessing import LabelEncoder
official_df=des[['company','segment']]
le=LabelEncoder()
le_class=le.fit_transform(official_df['segment'])
#word_class=official_df['official_class']
official_df['segment']=le_class
combiner=combine_models()
all_df=[official_df]+output
all_cluster=combiner.get_combine_cluster(all_df)
all_cluster.head()

[autoreload of jedi.api failed: Traceback (most recent call last):
  File "d:\anaconda\envs\ml\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "d:\anaconda\envs\ml\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "d:\anaconda\envs\ml\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "d:\anaconda\envs\ml\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "d:\anaconda\envs\ml\lib\site-packages\jedi\api\__init__.py", line 35, in <module>
    from jedi.inference.helpers import get_module_names, infer_call_of_leaf
ImportError: cannot import name 'get_module_names' from 'jedi.infere

[autoreload of jedi.inference.names failed: Traceback (most recent call last):
  File "d:\anaconda\envs\ml\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "d:\anaconda\envs\ml\lib\site-packages\IPython\extensions\autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "d:\anaconda\envs\ml\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "d:\anaconda\envs\ml\lib\site-packages\IPython\extensions\autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "d:\anaconda\envs\ml\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "d:\anaconda\envs\ml\lib\site-packages\IPython\extensions\autoreload.py", line 266, in update_function
    setattr(old, name, getattr(new, name))
ValueError: goto() requires a code object with 2 free vars, not 0
]
[au

,company,segment,bag of words,bert,pvdm 50 length,pvdbow 50 length,pvdm 100 length,pvdbow 100 length,pvdm 150 length,pvdbow 150 length,...,lsi 200 length,lsi 300 length,lsi 400 length,lsi 500 length,lsi 600 length,lsi 700 length,lsi 800 length,lsi 900 length,lsi 1000 length,lsi 1100 length
0,1418.T,7,1,7,7,11,2,1,6,2,...,10,3,3,1,2,0,11,4,7,0
1,1436.T,7,7,2,2,11,7,1,1,2,...,2,1,6,9,9,7,9,1,7,8
2,2122.T,1,9,7,0,11,6,1,8,2,...,10,0,5,2,3,8,11,4,0,0
3,2124.T,7,6,1,4,11,8,1,4,4,...,10,10,7,0,2,0,1,4,5,0
4,2136.T,7,10,7,7,11,2,1,6,2,...,0,5,3,5,10,0,0,8,0,8


In [161]:
from new_combine_models import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [162]:
performance=performance_analysis(class_df=all_cluster,return_df=stock_data)

In [163]:
performance.get_daliy_return()

D:\OneDrive - National University of Singapore\industry-classification-master\new_combine_models.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.return_map[name][cl].dropna(1, thresh=20, inplace=True)
d:\anaconda\envs\ml\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
D:\OneDrive - National University of Singapore\industry-classification-master\new_combine_models.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [165]:
performance.get_statistical_discribe()

In [166]:
performance.statistical_map['lsi 200 length'][11]

,cluster 0,cluster 1,cluster 2,cluster 3,cluster 4,cluster 5,cluster 6,cluster 7,cluster 8,cluster 9,cluster 10,cluster 11,most_positive_related,most_negative_related,isBiggest,R2
company,,,,,,,,,,,,,,,,
1840.T,0.530668,-0.0237142,-0.562006,0.131973,0.0990542,-0.822459,-0.609934,-1.17384,-0.133227,-0.83617,2.2958,2.28844,cluster 10,cluster 7,False,0.206454
1873.T,-0.617961,0.00863253,-0.192385,0.0894962,-0.71192,0.245342,0.223316,0.126922,0.209833,0.326568,0.70199,0.785238,cluster 11,cluster 4,True,0.461962
8999.T,0.429355,-0.0211998,0.305129,0.0115748,0.562564,0.119046,0.0548955,-0.166615,0.295845,-0.567025,-0.503175,0.740262,cluster 11,cluster 9,True,0.433938
1736.T,-0.211309,0.00558936,-0.630805,-0.121086,-0.0204752,0.481848,0.103107,0.215424,0.273848,-0.292319,-0.22471,0.859623,cluster 11,cluster 2,True,0.086167
1911.T,0.583849,0.0112129,0.0828924,-0.0383538,-0.156755,-0.0568119,0.135978,0.80015,-0.382656,0.429949,-1.11936,0.826678,cluster 11,cluster 10,True,0.516244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969.T,0.40422,0.0115573,0.38673,-0.0494507,0.322834,-0.404069,0.036546,-0.398379,0.0368039,0.304049,-0.466548,0.925578,cluster 11,cluster 10,True,0.556713
9503.T,-1.55181,0.000701332,1.00278,-0.0564451,0.228157,-0.880588,0.00400423,0.193959,0.225504,0.892879,0.479733,0.484777,cluster 2,cluster 0,False,0.353786
9504.T,-0.647196,0.00041421,0.724525,0.22709,0.617996,-0.648735,-0.0763507,-0.0989221,-0.174171,0.333681,0.105627,0.205234,cluster 2,cluster 5,False,0.388182


In [167]:
performance.print_table()

summary table:


,R2,proportion of right classification,classes number
segment,0.347233,0.506757,11.0
lsi 200 length,0.341915,0.459942,12.0
pvdm 100 length,0.338534,0.21332,12.0
pvdm 250 length,0.337828,0.42471,12.0
pvdm 200 length,0.334731,0.431467,12.0
bert,0.331638,0.388031,7.0
pvdm 150 length,0.331584,0.216216,11.0
pvdm 300 length,0.33068,0.235521,12.0
lsi 300 length,0.330075,0.488417,12.0
pvdm 350 length,0.329115,0.353764,12.0
